- Salma Elbess
- 201601152

##Data import

In [141]:
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np
import math
import pandas as pd

In [142]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [143]:
df = pd.read_csv('recipes_serp_data.csv', encoding='latin-1') #check encodings types
df.drop_duplicates(subset='snippet',inplace=True,ignore_index=True)

In [144]:
df.head()

,searchTerms,rank,title,snippet,displayLink,link,queryTime,totalResults,cacheId,count,cseName,cx,fileFormat,formattedSearchTime,formattedTotalResults,formattedUrl,htmlFormattedUrl,htmlSnippet,htmlTitle,inputEncoding,kind,mime,outputEncoding,pagemap,safe,searchTime,startIndex
0,dish recipe,1,Recipes - Dish,Chicken is a versatile ingredient that can be ...,dish.co.nz,https://dish.co.nz/recipes/,2019-03-30 01:15:35.167457+00:00,1090000000,8UR9TW0uG8QJ,10,PySearch,012859022920491477448:pubdbfjmmec,NaN,0.18,"1,090,000,000",https://dish.co.nz/recipes/,https://<b>dish</b>.co.nz/<b>recipes</b>/,Chicken is a versatile ingredient that can be ...,<b>Recipes</b> - <b>Dish</b>,utf8,customsearch#result,NaN,utf8,"{'cse_thumbnail': [{'width': '310', 'height': ...",off,0.182105,1
1,dish recipe,2,Easy Main Dish Recipes | Taste of Home,"No matter when I eat it, this tangy chicken di...",www.tasteofhome.com,https://www.tasteofhome.com/cooking-style/easy...,2019-03-30 01:15:35.167457+00:00,1090000000,-YQNzFg49T0J,10,PySearch,012859022920491477448:pubdbfjmmec,NaN,0.18,"1,090,000,000",https://www.tasteofhome.com/...recipes/easy-ma...,https://www.tasteofhome.com/...<b>recipes</b>/...,"No matter when I eat it, this tangy chicken <b...",Easy Main <b>Dish Recipes</b> | Taste of Home,utf8,customsearch#result,NaN,utf8,"{'cse_thumbnail': [{'width': '225', 'height': ...",off,0.182105,1
2,dish recipe,3,Main Dish Recipes - Allrecipes.com,Hundreds of main dish recipes. Choose from top...,www.allrecipes.com,https://www.allrecipes.com/recipes/80/main-dish/,2019-03-30 01:15:35.167457+00:00,1090000000,GFCWqkFsgBEJ,10,PySearch,012859022920491477448:pubdbfjmmec,NaN,0.18,"1,090,000,000",https://www.allrecipes.com/recipes/80/main-dish/,https://www.all<b>recipes</b>.com/<b>recipes</...,Hundreds of main <b>dish recipes</b>. Choose f...,Main <b>Dish Recipes</b> - Allrecipes.com,utf8,customsearch#result,NaN,utf8,"{'cse_thumbnail': [{'width': '170', 'height': ...",off,0.182105,1
3,dish recipe,4,Let's Dish Recipes | Family friendly recipes f...,"Fast, family friendly recipes that won't leave...",letsdishrecipes.com,https://letsdishrecipes.com/,2019-03-30 01:15:35.167457+00:00,1090000000,HO5gMe_HFsoJ,10,PySearch,012859022920491477448:pubdbfjmmec,NaN,0.18,"1,090,000,000",https://letsdishrecipes.com/,https://lets<b>dishrecipes</b>.com/,"Fast, family friendly <b>recipes</b> that won&...",Let&#39;s <b>Dish Recipes</b> | Family friendl...,utf8,customsearch#result,NaN,utf8,"{'website': [{'name': ""Let's Dish Recipes"", 'u...",off,0.182105,1
4,dish recipe,5,Italian Hot Dish Recipe | Taste of Home,My husband had a poor perception of healthy fo...,www.tasteofhome.com,https://www.tasteofhome.com/recipes/italian-ho...,2019-03-30 01:15:35.167457+00:00,1090000000,lmrSo2pYvHsJ,10,PySearch,012859022920491477448:pubdbfjmmec,NaN,0.18,"1,090,000,000",https://www.tasteofhome.com/recipes/italian-ho...,https://www.tasteofhome.com/<b>recipes</b>/ita...,My husband had a poor perception of healthy fo...,Italian Hot <b>Dish Recipe</b> | Taste of Home,utf8,customsearch#result,NaN,utf8,"{'cse_thumbnail': [{'width': '225', 'height': ...",off,0.182105,1


In [145]:
#copy data in another column
df['original_snippets'] = df['snippet']

##1.Perform basic text processing steps:

In [146]:
#process snippets
df['snippet'] = df['snippet'].astype(str)
df['snippet'] = df.snippet.apply(lambda x: x.lower())
df['snippet'] = df.snippet.apply(lambda x: word_tokenize(x))
#remove stop words
stop_words = set(stopwords.words('english'))
df['snippet'] = df.snippet.apply(lambda x: [token for token in x if token not in stop_words]) 

#remove panctuation 
df['snippet'] = df.snippet.apply(lambda x:[token for token in x if token.isalnum()]) 

#stemming 
stemmer = PorterStemmer()
df['snippet'] = df.snippet.apply(lambda x:[stemmer.stem(token) for token in x]) 


# lemmatization 
lemmatizer = WordNetLemmatizer()
df['snippet'] = df.snippet.apply(lambda x:[lemmatizer.lemmatize(token) for token in x]) 

In [150]:
#sample output
df['snippet'][0] 

['chicken',
 'versatil',
 'ingredi',
 'use',
 'array',
 'dish',
 'quick',
 'easi',
 'salad',
 'roast',
 'recip',
 'everi',
 'night',
 'week']

##2. Create Inverted Indexer:

In [147]:
length = df['snippet'].shape[0]

In [148]:
inverted_index = dict()
for i in range(df['snippet'].shape[0]):
    for j in range(len(df['snippet'][i])):
        addition_arr = np.zeros(length)
        addition_arr[i] = 1
        inverted_index[df['snippet'][i][j]] = inverted_index.get(df['snippet'][i][j],np.zeros(length)) + addition_arr

##3. Replace each entry with TF-IDF score:

In [151]:
for x, y in inverted_index.items():
    idf = math.log(sum((y !=0)*1)/sum(y[y!=0]))
    new_y = np.log(y+1)
    new_y = new_y*idf
    inverted_index[x] = new_y

##4.Simulate search engine process

In [ ]:
#tested query
query = 'Chicken dish'

In [160]:
#user input query
print('Enter query:')
query = input()

Enter query:
Chicken dish


In [161]:

#query parsing
#lower case
lowercase_query = query.lower()

#tokenize
tokens = word_tokenize(lowercase_query)

#remove stop words
stop_words = set(stopwords.words('english')) 
new_tokens = [token for token in tokens if token not in stop_words]

#remove panctuation 
clear_tokens = [token for token in new_tokens if token.isalnum()]

#stemming 
stemmer = PorterStemmer()
stemmed_tokens = [stemmer.stem(token) for token in clear_tokens]

# lemmatization 
lemmatizer = WordNetLemmatizer()
lemmatized_tokens = [lemmatizer.lemmatize(token) for token in stemmed_tokens]

print('Parsed query Tokens: ', lemmatized_tokens)

Parsed query Tokens:  ['chicken', 'dish']


In [158]:
#get query score for all snippets
all_words = inverted_index.keys()
scores = np.zeros(length)
for word in lemmatized_tokens:
    if word in all_words:
        scores = scores + np.abs(inverted_index[word])
most_likely_snippets = scores.argsort()[-3:][::-1]

In [168]:
pd.options.display.max_colwidth = 100

In [167]:
print('snippets with highest score')
for i in range(3):
    print('snippet ',str(i+1),":")
    print(df['original_snippets'][most_likely_snippets[i]])

snippets with highest score
snippet  1 :
Poulet Ã  la moambÃ© or simply poulet moambe is a savoury chicken dish popular 
in Central Africa and sometimes considered the national dish of the Democratic 
Republic of the Congo. The dish itself is made by combining chicken, spices and 
palm butter (moambÃ©) to create a stew-like consistency.
snippet  2 :
Mar 25, 2013 ... Chicken Machboos is the national dish of Bahrain. ... Chicken Machboos (
Bahraini Spiced Chicken and Rice) ... Let's make some Machboos!
snippet  3 :
Filipino Adobo Chicken â Chicken braised in vinegar and soy sauce with lots of 
garlic. ... More of my favorite, easy to make chicken dishes are Braised ChickenÂ ...
